In [23]:
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import mean_absolute_error

In [24]:
class CatBoost:
    def __init__(self, data):
        
        self.A = data.A
        self.B = data.B
        self.C = data.C
        
        self.model_a = CatBoostRegressor(iterations=1000,
                           depth=10,
                           learning_rate=0.01838436853816393,
                           task_type="GPU",
                           devices='0:1',
                           eval_metric="MAE",
                           random_seed=42)
        
        self.model_b = CatBoostRegressor(iterations=1000,
                           depth=10,
                           learning_rate=0.013526644801686614,
                           task_type="GPU",
                           devices='0:1',
                           eval_metric="MAE",
                           random_seed=42)
        
        self.model_c = CatBoostRegressor(iterations=1000,
                           depth=10,
                           learning_rate=0.06913412566825089,
                           task_type="GPU",
                           devices='0:1',
                           eval_metric="MAE",
                           random_seed=42)
        
    def modelfit(self, model, building):
     
        
        model.fit(building.train_x, building.train_y,
            eval_set=(building.val_x, building.val_y),
            early_stopping_rounds=50,
            use_best_model=True)

        #Predict training set:
        dtrain_predictions = model.predict(building.train_x)
        dval_predictions = model.predict(building.val_x)

        mae = mean_absolute_error(dtrain_predictions, building.train_y)
        mae_val = mean_absolute_error(dval_predictions, building.val_y)
        print(mae, mae_val)
        
        return mae, mae_val

        
    def fit(self):
        mae_a, mae_a_val = self.modelfit(self.model_a, self.A)
        mae_b, mae_b_val =self.modelfit(self.model_b, self.B)
        mae_c, mae_c_val =self.modelfit(self.model_c, self.C)
        
        mae_avg = (mae_a + mae_b + mae_c) / 3
        mae_val_avg = (mae_a_val + mae_b_val + mae_c_val) / 3
        print(f'Average MAE: {mae_avg}, Average val MAE: {mae_val_avg}')
    
    def predict(self):
        preds_a_train = np.absolute(self.model_a.predict(self.A.train_x))
        preds_a_val = np.absolute(self.model_a.predict(self.A.val_x))
        preds_a_test = np.absolute(self.model_a.predict(self.A.test_x))

        preds_b_train = np.absolute(self.model_b.predict(self.B.train_x))
        preds_b_val = np.absolute(self.model_b.predict(self.B.val_x))
        preds_b_test = np.absolute(self.model_b.predict(self.B.test_x))

        preds_c_train = np.absolute(self.model_c.predict(self.C.train_x))
        preds_c_val = np.absolute(self.model_c.predict(self.C.val_x))
        preds_c_test = np.absolute(self.model_c.predict(self.C.test_x))
        
        
        return [
            [preds_a_train, preds_a_val, preds_a_test],
            [preds_b_train, preds_b_val, preds_b_test],
            [preds_c_train, preds_c_val, preds_c_test],
        ]